## Genetic

Bước 1: Tiền xử lý dữ liệu ảnh. Lấy tên tệp và gán nó làm label cho tất cả các ảnh trong tệp đó. Chỉ sử dụng những ảnh đuôi .pgm.

In [1]:
import os
import cv2

ROOT_DIR = os.path.abspath("../")
data_dir = os.path.join(ROOT_DIR, "CMU_FACE_Data/faces")
X = []  # Features
y = []  # Labels
size = (64, 64)

for folder_name in os.listdir(data_dir):
    label = folder_name
    folder_path = os.path.join(data_dir, folder_name)
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, size)
        if image is not None:
            X.append(image)
            y.append(label)

In [2]:
import numpy as np

print(np.unique(y))

['an2i' 'at33' 'boland' 'bpm' 'ch4f' 'cheyer' 'choon' 'danieln' 'glickman'
 'karyadi' 'kawamura' 'kk49' 'megak' 'mitchell' 'night' 'phoebe' 'saavik'
 'steffi' 'sz24' 'tammo']


Bước 2: Làm phẳng dữ liệu ảnh và chuyển sang Numpy array dạng float.

In [3]:
data = []
for x in X:
    d = x.flatten()
    data.append(d)
data = np.array(data)
data = data.astype(float)

Bước 3: Sử dụng phương thức train_test_split để chia dữ liệu thành tập huấn luyện và tập kiểm tra với tỉ lệ 8 : 2.

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)

Bước 4:
1. Tạo một đối tượng clf_rf từ lớp RandomForestClassifier với tham số max_depth=6 và random_state=42. Đối tượng này sẽ được sử dụng làm bộ phân loại trong quá trình tìm kiếm di truyền.
2. Tạo một đối tượng selector từ lớp GeneticSelectionCV với các tham số như sau:
    + clf_rf: Bộ phân loại RandomForest đã được khởi tạo trước đó.
    + cv=5: Số lượng folds trong cross-validation.
    + verbose=0: Không hiển thị thông báo trong quá trình tìm kiếm.
    + scoring="accuracy": Đánh giá hiệu suất sử dụng độ chính xác.
    + max_features=10: Số lượng đặc trưng tối đa được chọn trong mỗi thế hệ.
    + n_population=50: Số lượng cá thể trong quần thể di truyền.
    + crossover_proba=0.5: Xác suất chéo gen trong quá trình lai ghép.
    + mutation_proba=0.2: Xác suất đột biến gen trong quá trình đột biến.
    + n_generations=50: Số lượng thế hệ trong quá trình tìm kiếm.
    + crossover_independent_proba=0.5: Xác suất lai ghép độc lập giữa các cặp gen trong quá trình lai ghép.
    + mutation_independent_proba=0.04: Xác suất đột biến độc lập của từng gen trong quá trình đột biến.
    + tournament_size=5: Số lượng cá thể được chọn ngẫu nhiên để tham gia giải đấu (tournament selection).
    + n_gen_no_change=10: Số lượng thế hệ liên tiếp mà không có sự cải thiện trong điểm số tốt nhất.
    + caching=True: Lưu trữ các giá trị trung gian để tăng tốc độ tính toán.
    + n_jobs=-1: Sử dụng tất cả các CPU trong quá trình tính toán.
3. Sử dụng phương thức fit(X_train, y_train) trên đối tượng selector để thực hiện quá trình tìm kiếm di truyền trên dữ liệu huấn luyện X_train và y_train.
4. Sử dụng phương thức predict(X_test) trên đối tượng selector để dự đoán nhãn cho tập kiểm tra X_test. Kết quả được lưu vào biến y_pred.

In [5]:
from sklearn.ensemble import RandomForestClassifier
from genetic_selection import GeneticSelectionCV

clf_rf = RandomForestClassifier(max_depth=6, random_state=42)

selector = GeneticSelectionCV(
    clf_rf, cv=5, verbose=0,
    scoring="accuracy", max_features=10,
    n_population=50, crossover_proba=0.5,
    mutation_proba=0.2, n_generations=50,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.04,
    tournament_size=5, n_gen_no_change=10,
    caching=True, n_jobs=-1)
selector = selector.fit(X_train, y_train)

In [6]:
y_pred = selector.predict(X_test)

d:\Tools\Python3.9\lib\site-packages\sklearn\utils\metaestimators.py:121: FutureWarning: if_delegate_has_method was deprecated in version 1.1 and will be removed in version 1.3. Use available_if instead.
  warnings.warn(


Bước 5: Tính Accuracy và F1 Score.

+ Độ chính xác (Accuracy): Độ chính xác được tính bằng cách so sánh nhãn thực tế (y_test) với nhãn dự đoán (y_pred) trên tập kiểm tra. Độ chính xác là tỷ lệ giữa số lượng dự đoán chính xác và tổng số mẫu. acc = TP + TN / (TP + TN + FP + FN)
+ F1 Score: F1 Score là một số đo kết hợp giữa độ chính xác (precision) và độ phủ (recall) của mô hình. F1 Score là trung bình điều hoà của precision và recall, và được tính bằng công thức: F1 = 2 * (precision * recall) / (precision + recall).
    Precision = TP / (TP + FP)
    Recall = TP / (TP + FN)

In [7]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9788359788359788


In [8]:
from sklearn.metrics import f1_score

f1 = f1_score(y_test, y_pred, average='macro')
print("F1 Score:", f1)

F1 Score: 0.9774236237898345
